In [2]:
import pandas as pd
import sqlite3 as sql
import warnings
import pyodbc
warnings.simplefilter('ignore')

In [3]:
connection_sales = sql.connect('go_sales_train.sqlite')
connection_crm = sql.connect('go_crm_train.sqlite')
connection_staff = sql.connect('go_staff_train.sqlite')

empty tables

In [4]:

# Maak verbinding met SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=MSI\\SQLEXPRESS;'  # Gebruik de juiste servernaam uit de screenshot
    'DATABASE=sdm;'  # Gebruik de database 'sdm'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Get all tables with constraints
tables_query = """
SELECT DISTINCT OBJECT_NAME(parent_object_id) AS TableName
FROM sys.foreign_keys
"""

try:
    cursor.execute(tables_query)
    tables_with_constraints = [row[0] for row in cursor.fetchall()]
    
    # Disable constraints for each table individually
    for table in tables_with_constraints:
        print(f"  Disabling constraints for {table}...")
        cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
        conn.commit()
    
    print("All constraints disabled")
except Exception as e:
    print(f"Error disabling constraints: {str(e)}")
    # Continue anyway - we'll still try to import data

# Get all tables
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
all_tables = [row[0] for row in cursor.fetchall()]

# Empty each table
for table_name in all_tables:
    print(f" Legen van tabel: {table_name}")
    try:
        cursor.execute(f"DELETE FROM [{table_name}];")
        conn.commit()
    except Exception as e:
        print(f"Error deleting from {table_name}: {str(e)}")

# Re-enable constraints
try:
    # Re-enable constraints for each table individually
    for table in tables_with_constraints:
        print(f"  Re-enabling constraints for {table}...")
        cursor.execute(f"ALTER TABLE [{table}] WITH CHECK CHECK CONSTRAINT ALL")
        conn.commit()
    
    print("All constraints re-enabled successfully")
except Exception as e:
    print(f"Error re-enabling constraints: {str(e)}")

print(" Alle tabellen in de database 'sdm' zijn geleegd.")

# Close connections at the very end
cursor.close()
conn.close()

  Disabling constraints for COUNTRY...
  Disabling constraints for INVENTORY_LEVELS...
  Disabling constraints for ORDER_DETAILS...
  Disabling constraints for ORDER_HEADER...
  Disabling constraints for PRODUCT...
  Disabling constraints for PRODUCT_FORECAST...
  Disabling constraints for PRODUCT_TYPE...
  Disabling constraints for RETAILER...
  Disabling constraints for RETAILER_CONTACT...
  Disabling constraints for RETAILER_HEADQUARTERS...
  Disabling constraints for RETAILER_SITE...
  Disabling constraints for RETURNED_ITEM...
  Disabling constraints for SALES_BRANCH...
  Disabling constraints for SALES_DEMOGRAPHIC...
  Disabling constraints for SALES_STAFF...
  Disabling constraints for SATISFACTION...
  Disabling constraints for TRAINING...
All constraints disabled
 Legen van tabel: AGE_GROUP
 Legen van tabel: COUNTRY
 Legen van tabel: COURSE
 Legen van tabel: ORDER_DETAILS
 Legen van tabel: ORDER_HEADER
 Legen van tabel: ORDER_METHOD
 Legen van tabel: PRODUCT
 Legen van tabel: 

importeer data

In [ ]:

# Helper function to convert NumPy types to Python native types
def convert_numpy_type(val):
    if pd.isnull(val):
        return None
    elif hasattr(val, 'item'):  # Handles numpy types (int64, float64, etc.)
        return val.item()
    else:
        return val

# Connect to SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=MSI\\SQLEXPRESS;DATABASE=SDM;Trusted_Connection=yes;')
cursor = conn.cursor()

# Disable constraints
cursor.execute("SELECT DISTINCT OBJECT_NAME(parent_object_id) FROM sys.foreign_keys")
tables = [row[0] for row in cursor.fetchall()]
for table in tables:
    cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
conn.commit()

# SQLite imports
sqlite_files = [
    {"db": "go_sales_train.sqlite", "tables": ["product", "order_details", "sales_branch", "returned_item","sales_staff"]},
    {"db": "go_staff_train.sqlite", "tables": ["training", "course", "satisfaction", "satisfaction_type"]},
    {"db": "go_crm_train.sqlite", "tables": ["country"]}
]

for file in sqlite_files:
    conn_sqlite = sql.connect(file['db'])
    for table in file["tables"]:
        df = pd.read_sql(f"SELECT * FROM {table}", conn_sqlite)
        if df.empty: continue
        
        cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table}'")
        sql_cols = [row[0] for row in cursor.fetchall()]
        common_cols = [col for col in df.columns if col in sql_cols]
        
        cols = ", ".join([f"[{col}]" for col in common_cols])
        placeholders = ", ".join(["?"] * len(common_cols))
        
        for _, row in df[common_cols].iterrows():
            try:
                cursor.execute(f"INSERT INTO {table} ({cols}) VALUES ({placeholders})", tuple(row))
            except Exception as e:
                pass  # Skip errors
        conn.commit()
    conn_sqlite.close()

# CSV imports with specific column handling
csv_files = {
    "product_forecast_train.csv": {
        "table": "PRODUCT_FORECAST",
        "mapping": {
            "`PRODUCT_NUMBER": "PRODUCT_NUMBER",
            "YEAR": "YEAR",
            "MONTH": "MONTH",
            "EXPECTED_VOLUME": "EXPECTED_VOLUME"
        }
    },
    "inventory_levels_train.csv": {
        "table": "INVENTORY_LEVELS",
        "mapping": {
            "Unnamed: 0": "ID",
            "INVENTORY_YEAR": "INVENTORY_YEAR",
            "INVENTORY_MONTH": "INVENTORY_MONTH",
            "PRODUCT_NUMBER": "PRODUCT_NUMBER",
            "INVENTORY_COUNT": "INVENTORY_COUNT"
        }
    }
}

for csv_file, info in csv_files.items():
    table_name = info["table"]
    column_mapping = info["mapping"]
    
    print(f"Importing {csv_file} to {table_name}")
    df = pd.read_csv(csv_file)
    
    # Rename columns according to mapping
    rename_dict = {}
    for csv_col, sql_col in column_mapping.items():
        if csv_col in df.columns:
            rename_dict[csv_col] = sql_col
    
    if rename_dict:
        df = df.rename(columns=rename_dict)
    
    # Get SQL Server columns
    cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'")
    sql_cols = [row[0] for row in cursor.fetchall()]
    
    # Get common columns
    common_cols = [col for col in df.columns if col in sql_cols]
    
    if common_cols:
        cols_str = ", ".join([f"[{col}]" for col in common_cols])
        placeholders = ", ".join(["?"] * len(common_cols))
        
        for _, row in df.iterrows():
            try:
                # Convert NumPy types to Python native types
                values = [convert_numpy_type(row[col]) for col in common_cols]
                cursor.execute(f"INSERT INTO {table_name} ({cols_str}) VALUES ({placeholders})", tuple(values))
            except Exception as e:
                print(f"Error inserting row: {e}")  # Print errors for debugging
        
        conn.commit()
        print(f"Successfully imported {csv_file}")
    else:
        print(f"No matching columns found for {table_name}")

# Re-enable constraints with NOCHECK
for table in tables:
    cursor.execute(f"ALTER TABLE [{table}] WITH NOCHECK CHECK CONSTRAINT ALL")
conn.commit()

cursor.close()
conn.close()

Importing product_forecast_train.csv to PRODUCT_FORECAST
Successfully imported product_forecast_train.csv
Importing inventory_levels_train.csv to INVENTORY_LEVELS
Successfully imported inventory_levels_train.csv
